# Update records on GEOMG via script

The `update.ipynb` is a script aims to modify fields' values on GEOMG via python. We used to **download CSV files -> modify values locally -> upload again** to modify a large number of datasets, or **open each dataset page -> modify values -> update and open next data page** to modify datasets one by one. However, this script offers you a third method for GEOMG document updates.




> Originally created by **Gene Cheng [(@Ziiiiing)](https://github.com/Ziiiiing)** on **Oct 3, 2021**

> Updated by **Gene Cheng [(@Ziiiiing)](https://github.com/Ziiiiing)** on **Oct 24, 2021**

In [1]:
# uncomment & run this cell if the 'mechanize' module is not installed yet

!pip install mechanize

In [2]:
import mechanize
import time
import csv

## Step 1: Prepare a CSV File

Store updated **values** with **field names** in a local CSV file under the same directory.
- First row should be **field names** 
- First column should be **IDs** for each document. 

Please look at the [README.md](https://github.com/BTAA-Geospatial-Data-Project/geomg-documents-update/blob/main/README.md) for more information.


In [17]:
# Hello, please edit here !!
csv_file = "/Users/Thenewsguy/Documents/GitHub/workflows/editing/2022-10-02_08-00-03uri_report.csv"

In [16]:
data = {}
with open(csv_file) as fr:
    reader = csv.reader(fr)
    fields = next(reader)[1:]
    for row in reader:
        ID = row[0]
        dictVal = {}
        for i in range(len(row)-1):
            nameAttr = fields[i]
            newVal = row[i+1]
            if newVal.startswith("[\'"):
                newVal = eval(newVal)
            dictVal[nameAttr] = newVal
        data[ID] = dictVal


## Step 2: User Login on GEOMG

After preparation, we are ready for interacting with the GEOMG. First thing first, you need to modify the value of `username` and `password` to your own ones for GEOMG login. Make sure your personal information is not exposed on the internet.

In [18]:
# Hello, please edit here !!
username = "danie861@umn.edu"
password = "ARDgeomg1994123?!"

In [19]:
# Perform login 
login_url = "https://geomg.lib.umn.edu/users/sign_in"

br = mechanize.Browser()
br.set_handle_robots(False)   # ignore robots

# browse the Login Page and select the right form for login
br.open(login_url)
br.select_form(nr=1)

# input and submit the username & password
br["user[email]"] = username
br["user[password]"] = password
br.submit()

# redirect if successfully logged in
if br.geturl() ==  login_url:
    print(">>> Failed to login.")
else:
    print('>>>> Successfully logged in.')


>>>> Successfully logged in.


## Step 3: Modify Web Contents Online

In [20]:
# iterate the 'modifies' dictionary and make updates
count = 0
nonexist = []
failed = []

for ID in data:
    count += 1
    item_url = "https://geomg.lib.umn.edu/documents/{}".format(ID)
    modifies = data[ID]

    try:
        br.open(item_url)          # open the edit page for each record
        br.select_form(nr=1)       # the index of the form is 1
    
        # iterate field&value pairs to modify
        for field, newval in modifies.items():
            br[field] = newval

        # submit the changes for this document
        br.submit()
        print(">>> [{}/{}] Updating {} .................... √".format(count, len(data), ID))
    
    # skip the nonexist record with error code 404 if any error occurs
    except mechanize.HTTPError as e:
        # ignore the non-exist records
        if e.code == 404:
            print(">>> [{}/{}] Updating {} .................... x".format(count, len(data), ID))
            nonexist.append(ID)
        else:
            print(">>> [{}/{}] Updating {} .................... x".format(count, len(data), ID))
            failed.append(ID)        # store failed item and try again later
    except:
        print(">>> [{}/{}] Updating {} .................... x".format(count, len(data), ID))
        failed.append(ID)

            
# print out the summary
print('\n-------------- Summary --------------')
print('Successful Updates: {}'.format(len(data)-len(nonexist)-len(failed)))
print('Datasets Not Exist: {}'.format(len(nonexist)))
print('Failed Updates: {}'.format(len(failed)))

if failed:
    print('\n-------------- Manual Edits Needed for Failed Updates --------------')
    for ID in failed:
        item_url = 'https://geomg.lib.umn.edu/documents/{}'.format(ID)
        print(item_url)

>>> [1/1076] Updating 2dab2f70653f4bb8b4f2b51619ec8329_0 .................... x
>>> [2/1076] Updating fe2f692918a04c13a6cead436e7eaec9_0 .................... x
>>> [3/1076] Updating aa3158d40cb6458c873075ce9437649c_0 .................... x
>>> [4/1076] Updating a210575930354d758c12d7f45eebaa2f_0 .................... x
>>> [5/1076] Updating 3c36523e359f4d45aee1f173d4b9df29_0 .................... x
>>> [6/1076] Updating 0825badfe6304620a998d162be0e135e_0 .................... x
>>> [7/1076] Updating f337e36f-83e0-4ca7-b47e-736d98d05ca5 .................... x
>>> [8/1076] Updating 6497acd8-e19a-4482-948f-3312ae16b634 .................... x
>>> [9/1076] Updating ddc18a58-6e43-4c5a-80f6-e0ac578e4652 .................... x
>>> [10/1076] Updating 5967807e44e445229d64b70d23081744_0 .................... x
>>> [11/1076] Updating 7cf4117186c7486290aace6528a24915&sublayer=2 .................... x
>>> [12/1076] Updating 798659bde6874043bee40491db5cd9a6&sublayer=11 .................... x
>>> [13/1076

>>> [107/1076] Updating 6m47-at6y .................... x
>>> [108/1076] Updating 6wtj-5znn .................... x
>>> [109/1076] Updating r3qj-2ifh .................... x
>>> [110/1076] Updating 77mk-gzh4 .................... x
>>> [111/1076] Updating 7g6t-cgv2 .................... x
>>> [112/1076] Updating 7u3e-affd .................... x
>>> [113/1076] Updating 7v76-hknd .................... x
>>> [114/1076] Updating 7y6i-3pcr .................... x
>>> [115/1076] Updating 873b-4xqf .................... x
>>> [116/1076] Updating 8s6d-nxcr .................... x
>>> [117/1076] Updating 937e-xrmk .................... x
>>> [118/1076] Updating 9bbz-8q88 .................... x
>>> [119/1076] Updating 9qgs-dngv .................... x
>>> [120/1076] Updating 9sax-si6k .................... x
>>> [121/1076] Updating a3qh-vp5x .................... x
>>> [122/1076] Updating a4sr-x5f7 .................... x
>>> [123/1076] Updating nsuz-8g9d .................... x
>>> [124/1076] Updating a4xu-jw

>>> [243/1076] Updating baltimore::mizod .................... x
>>> [244/1076] Updating baltimore::museums-1 .................... x
>>> [245/1076] Updating baltimore::neighborhoods .................... x
>>> [246/1076] Updating baltimore::ng-911-street-centerline .................... x
>>> [247/1076] Updating baltimore::nursing-homes .................... x
>>> [248/1076] Updating baltimore::parking-facilities .................... x
>>> [249/1076] Updating baltimore::parks-1 .................... x
>>> [250/1076] Updating baltimore::police-districts .................... x
>>> [251/1076] Updating baltimore::polling-place .................... x
>>> [252/1076] Updating baltimore::public-pools-in-baltimore-city-1 .................... x
>>> [253/1076] Updating baltimore::red-light-cameras-1 .................... x
>>> [254/1076] Updating baltimore::restaurants .................... x
>>> [255/1076] Updating 7qbt-2pwy .................... x
>>> [256/1076] Updating 8f6f-h228 .................... 

>>> [345/1076] Updating 6643-2qhy .................... x
>>> [346/1076] Updating egle::household-drug-take-back-locations .................... x
>>> [347/1076] Updating d2476e90a54c4e3cbb24b2c3cb53ef1b_27 .................... x
>>> [348/1076] Updating 1b0972f5b5d94bf0b1846e283776e5b3_2 .................... x
>>> [349/1076] Updating ca8m-ir6r .................... x
>>> [350/1076] Updating montcopa::montgomery-county-parcels-june-fs .................... x
>>> [351/1076] Updating 0782c58466f445fb9c8bd84f0e26944a_26 .................... x
>>> [352/1076] Updating 2517441b10854a03809a8afcf7735da2_16 .................... x
>>> [353/1076] Updating 9abb229411ce4acabf935eefde21f996_0 .................... x
>>> [354/1076] Updating 6jcd-xgn7 .................... x
>>> [355/1076] Updating 1bb071e094ee4b059ec1abd36f5f91c8_0 .................... x
>>> [356/1076] Updating egle::primary-sulfur-dioxide-national-ambient-air-quality-standard-nonattainment-areas-2010- .................... x
>>> [357/1076] 

>>> [446/1076] Updating fc5c1d0e-6a05-41f1-85da-5d4cc8b0a62d .................... x
>>> [447/1076] Updating 839318ce-5b52-4817-b37a-0826d10e08be .................... x
>>> [448/1076] Updating def8e1df-1e69-4f9e-9f18-cc961e6c7c6b .................... x
>>> [449/1076] Updating a61cc119-c84e-4e9a-998d-1c2bede3a388 .................... x
>>> [450/1076] Updating a70e60b3-a3e7-47b3-8587-c852d8932ae1 .................... x
>>> [451/1076] Updating 1f106bc2-941c-48a0-aa91-d1bdfd9e38b3 .................... x
>>> [452/1076] Updating d8b9373c-3993-4b54-9111-b0e1e90913db .................... x
>>> [453/1076] Updating dea6f016-9252-4b8e-a53c-a096f5fa24a2 .................... x
>>> [454/1076] Updating d5431503-8d83-4bbe-8c7a-e2cf33ce46b0 .................... x
>>> [455/1076] Updating cd0dadd6-7bec-4917-bb35-bb52080e6b5e .................... x
>>> [456/1076] Updating 5e28dfb3-58e0-4a72-9a0b-3af364ef69ef .................... x
>>> [457/1076] Updating f0099730-d4d4-4711-b810-b414b040a705 ...............

...

>>> [655/1076] Updating maryland::maryland-ecosystem-services-atmospheric-particulate-matter-2-5-removal-kg-per-year .................... x
>>> [656/1076] Updating maryland::maryland-ecosystem-services-atmospheric-sulfur-dioxide-removal-economic-benefit-dollar-per-year .................... x
>>> [657/1076] Updating maryland::maryland-ecosystem-services-atmospheric-sulfur-dioxide-removal-kg-per-year .................... x
>>> [658/1076] Updating maryland::maryland-ecosystem-services-flood-prevention-and-stormwater-mitigation-potential-economic-value-dollars-per-year .................... x
>>> [659/1076] Updating maryland::maryland-ecosystem-services-flood-prevention-and-stormwater-mitigation-potential-index .................... x
>>> [660/1076] Updating maryland::maryland-ecosystem-services-groundwater-recharge-economic-benefit-dollars-per-year .................... x
>>> [661/1076] Updating maryland::maryland-ecosystem-services-groundwater-recharge-m3-per-year .................... x
>>>

>>> [749/1076] Updating 38c600b8ccdd4c42a404c606e7b31e89 .................... x
>>> [750/1076] Updating df93f4aff4ea42ca995b43236c3b8369 .................... x
>>> [751/1076] Updating d66d6800-f1a4-4d11-b933-d95f5eb85260 .................... x
>>> [752/1076] Updating 6d6c8e9a-d7c2-480b-a0d2-f39b9fa4c298 .................... x
>>> [753/1076] Updating 55a08475-7129-4d1d-80eb-862cc5b985c6 .................... x
>>> [754/1076] Updating e469449d-3716-4eb0-a3c7-2299d2d925e5 .................... x
>>> [755/1076] Updating 0c51c70c-71a7-4f96-9461-c434dbe0b39e .................... x
>>> [756/1076] Updating 27964c540d634f1cb4a6967126d6ff28_16 .................... x
>>> [757/1076] Updating f2122360bfd741f09142ca9cf5de2e6f_21 .................... x
>>> [758/1076] Updating iowadnr::potentail-karst-areas .................... x
>>> [759/1076] Updating 1b3e3a79-2570-4027-b2e6-4d34ec04c4c8 .................... x
>>> [760/1076] Updating 927c2850-56c2-498e-bb4b-40d56179955c .................... x
>>> [761

>>> [853/1076] Updating 3755d2d08a2d413eba7a23f58293a110_0 .................... x
>>> [854/1076] Updating 3ce040f38b9841d2b6724fa89d8d6d93_0 .................... x
>>> [855/1076] Updating 50d820b1ccde4e58928bc5b8d77988ff_0 .................... x
>>> [856/1076] Updating 5ed04329add948e896788d380af8d764_0 .................... x
>>> [857/1076] Updating 93291aefbf334c51807aba595821d3ba_0 .................... x
>>> [858/1076] Updating a908d8d1b478482e9533b6f62e675a45_0 .................... x
>>> [859/1076] Updating af41c161248b4048a0596ed460d25ef2_0 .................... x
>>> [860/1076] Updating b81ad6323c244f139dce30027c3d312d_2 .................... x
>>> [861/1076] Updating d036ec526425465491b491403b656360_0 .................... x
>>> [862/1076] Updating d605a2c12241488cbab4fb85ca8df7ec_0 .................... x
>>> [863/1076] Updating e255e0e21495494fad78109a977ab267_0 .................... x
>>> [864/1076] Updating ee9ac3ce95874e438ffc931837d8fe67_0 .................... x
>>> [865/1076] U

>>> [950/1076] Updating DCGIS::topography-northwest-dc .................... x
>>> [951/1076] Updating DCGIS::topography-southeast-dc .................... x
>>> [952/1076] Updating DCGIS::topography-southwest-dc .................... x
>>> [953/1076] Updating DCGIS::voting-precinct-1990 .................... x
>>> [954/1076] Updating DCGIS::voting-precinct-2002 .................... x
>>> [955/1076] Updating DCGIS::voting-precinct-2008 .................... x
>>> [956/1076] Updating DCGIS::zoning-campus-areas .................... x
>>> [957/1076] Updating DCGIS::zoning-regulations-of-1958 .................... x
>>> [958/1076] Updating DCGIS::zoning-to-property-lot-query-layer .................... x
>>> [959/1076] Updating 4321436d39044423a0a93aefd897eadc_0 .................... x
>>> [960/1076] Updating 9077b99e6fbf4bba9e2db4a6d2594512_0 .................... x
>>> [961/1076] Updating 970cd4d72c384086948f385d2980482f_0 .................... x
>>> [962/1076] Updating a2d0744ac0554fa3a998a375d8c

>>> [1051/1076] Updating cc0d3125-cab9-4a80-8235-ccd9b25d6549 .................... x
>>> [1052/1076] Updating 55c1395a-1124-445c-8adf-f51c997bd75b .................... x
>>> [1053/1076] Updating f95151f1-e4db-461b-a396-e317aaff9935 .................... x
>>> [1054/1076] Updating db6670e0-477a-4a6a-9243-f7c95f6b127c .................... x
>>> [1055/1076] Updating f9b09dd4-077e-47fa-b924-01a0261c8606 .................... x
>>> [1056/1076] Updating b1900288-2ee3-4f64-a946-1273074fbb71 .................... x
>>> [1057/1076] Updating 4b7ddebf-6665-42c7-a006-62d54ac132c3 .................... x
>>> [1058/1076] Updating 0ee32ad2-79a0-436d-a484-cb68d4b32565 .................... x
>>> [1059/1076] Updating fe108791-ce15-4313-8635-11896142cbb4 .................... x
>>> [1060/1076] Updating 3087ec0e-7fbd-4ffd-a573-bf4764f9c3e9 .................... x
>>> [1061/1076] Updating 4affface-107e-4a3e-b08a-b3bae474fefd .................... x
>>> [1062/1076] Updating 00fcfa6a-d8e0-45d5-ba57-4f1fd83573b1 ...